# Neutrophil Chemotaxis Assay

## Autogenerated Report: EPIC Neutrophil Chemotaxis Assay Analysis
#### Website: https://github.com/AlphonsG/EPIC-BBox-Cell-Tracking

In [ ]:
import shutil
import sys
import os
import csv
from copy import deepcopy
import math
from itertools import chain
import random as rd
import scipy
from statistics import mean, pstdev

import cv2

from IPython.display import Image, Video

import matplotlib.pyplot as plt

import epic
from epic.analysis.cell_migration_analysis import (drnty_vs_euclid_dist,
                                                   metric_box_plots, metric_mthd_avgs,
                                                   smp_trks_le_dist, smp_trks_le_propn, 
                                                   smp_trks_len, traj_plots)
from epic.analysis.cell_migration_metrics import MetricFactory
from epic.utils.cell_migration import detect_leading_edges
from epic.utils.file_processing import (load_imgs, load_motc_dets, load_motc_tracks,
                                        save_imgs, save_motc_tracks, save_video,
                                        video_reshape)
from epic.utils.image_processing import draw_bounding_boxes, draw_leading_edges, draw_tracks
from epic.utils.misc import create_tracklets

In [ ]:
# preconfiguration
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100
rd.seed(0)
VID_FILENAME = 'video'
CSV_FILENAME = 'cell_counts.csv'
CELL_COUNTS_FIG_FILENAME = 'cell_counts.png'
DPI = 1000

In [ ]:
# start
cwd = os.getcwd()

## Image Series

In [ ]:
# load images
motchallenge_img_dir = os.path.join(cwd, '..', 'img1')
img_dir = (motchallenge_img_dir if os.path.isdir(motchallenge_img_dir) else 
           os.path.join(cwd, '..'))
assert os.path.isdir(img_dir), f'Image directory {img_dir} could not be found.'
imgs = load_imgs(img_dir) 
assert len(imgs) > 1, 'Raw image series shorter than 2 frames.'
print(f'Loaded {len(imgs)} images.')

In [ ]:
for i, img in enumerate(imgs, start=1):
    print(f'Image {i}: {img[0]}')

In [ ]:
# show image series
output_dir = os.path.join(cwd, 'Raw_Image_Series')
if os.path.isdir(output_dir):
    shutil.rmtree(output_dir)
os.mkdir(output_dir)
save_imgs(imgs, output_dir)
raw_vid = os.path.join(output_dir, VID_FILENAME)
save_video(imgs, raw_vid)
raw_vid +=  epic.VID_FILE_EXT
dsp_wdh, dsp_hgt = video_reshape(raw_vid, 500)
Video(raw_vid, embed=True, width=dsp_wdh, height=dsp_hgt, html_attributes='controls loop autoplay')

## Cell Counts

In [ ]:
# PARAMETER - None OR (> 0 AND <= 1)
dets_min_score = 0.35  # only load detections with this minimum confidence score

In [ ]:
# load detections
motc_dets_file = os.path.join(os.getcwd(), '..', epic.DETECTIONS_DIR_NAME, 
                              epic.MOTC_DETS_FILENAME)
assert os.path.isfile(motc_dets_file), f'MOTC detections file {motc_dets_file} could not be found.'
dets = load_motc_dets(motc_dets_file, dets_min_score)
assert len(dets) > 1, 'Detections found in less than 2 frames.'
num_frames = min(len(imgs), len(dets))
imgs, dets = imgs[0: num_frames], dets[0: num_frames]
dets = create_tracklets(dets, imgs)
print(f'Loaded {len(list(chain.from_iterable(list(dets))))} detections.')

In [ ]:
# show detections
dets_imgs = deepcopy(imgs)
for ds in dets:
    draw_bounding_boxes(ds, dets_imgs, colour=(0, 0, 255))
output_dir = os.path.join(cwd, 'All_Detections')
if os.path.isdir(output_dir):
    shutil.rmtree(output_dir)
os.mkdir(output_dir)
save_imgs(dets_imgs, output_dir)
dets_vid = os.path.join(output_dir, VID_FILENAME)
save_video(dets_imgs, dets_vid)
dets_vid += epic.VID_FILE_EXT
dsp_wdh, dsp_hgt = video_reshape(dets_vid, 500)
Video(dets_vid, embed=True, width=dsp_wdh, height=dsp_hgt, html_attributes='controls loop autoplay')

In [ ]:
# print cell counts and save as csv
try:
    with open(CSV_FILENAME, 'w', newline='') as csv_file:
        fields = ['Frame', 'Number of Cells']
        writer = csv.writer(csv_file)
        writer.writerow(fields)
        num_dets = []
        for i, ds in enumerate(dets, start=1):
            num_dets.append(len(ds))
            writer.writerow([i, len(ds)])
            print(f'Frame {i} contains {len(ds)} cells')      
except (OSError, csv.Error, ValueError) as e:
    msg = 'Could not create stats file. Message: {}.'.format(str(e))
    warnings.warn(msg, UserWarning)
                             
stats = scipy.stats.describe(num_dets, axis=None)
print(f'\nMinimum number of cells per image: {stats[1][0]}')
print(f'Maximum number of cells per image: {stats[1][1]}')
print(f'Average number of cells per image: {round(stats[2], 2)} '
      f'(Standard Deviation: {round(math.sqrt(stats[3]), 2)})')                        

In [ ]:
# show cell counts figure
times = list(range(1, len(num_dets) + 1))
plt.figure()
plt.scatter(times, num_dets)
plt.title('Cells Counts')
plt.xlabel('Frame Number (Frame)')
plt.ylabel('Number of Cells')
plt.ylim(0, max(num_dets) * 1.5)
plt.savefig(CELL_COUNTS_FIG_FILENAME, dpi=DPI, bbox_inches=0)
plt.close(plt.gcf())
Image(CELL_COUNTS_FIG_FILENAME, width=750)